# 🔬 tRNA-Sec Structural Exploration Notebook

This Jupyter notebook will guide you through an exploration of the structure and sequences of tRNA^Sec, the molecule responsible for carrying selenocysteine during protein synthesis.

The workflow includes:

- Loading sequences from your local multifasta file (tRNA_sequences.fasta).
- Comparing human tRNA^Sec with that of a bacterium (E. coli) and an archaeon (M. kandleri).
- Fetching PDB structures.
- Analyzing secondary and tertiary features.
- Visualizing structures in 3D.

## Section 1: Setup
First, you need to install the necessary libraries. Run the following command to install Biopython, py3Dmol, and requests.

In [1]:
%pip install biopython py3Dmol requests

Note: you may need to restart the kernel to use updated packages.


Now, import the libraries you'll use in the following sections.



In [2]:
from Bio import SeqIO
from Bio.Align import PairwiseAligner
from Bio.PDB import PDBParser
import requests
import py3Dmol
import re
import pandas as pd
from Bio.PDB import PDBList
import os

## Section 2: Loading Sequences from Your Local File - Extract metadata with regex

We'll use the `Biopython` library to parse your multifasta file, `tRNA_sequences.fasta`, and print the first 10 sequences to verify that the file has been loaded correctly.

In [3]:
fasta_path = "../data/processed/tRNA_sequences.fasta"
sequences = list(SeqIO.parse(fasta_path, "fasta"))


print(f"Loaded {len(sequences)} sequences")


Loaded 400 sequences


Now, let's store specific sequences from the file into variables for later use.



In [4]:
records = []
pdb_pattern = re.compile(r"PDB\s+(\w{4})", re.IGNORECASE)

for s in sequences:
    desc = s.description
    urs = s.id
    match = pdb_pattern.search(desc)
    pdb_id = match.group(1).upper() if match else None

    # Try to extract species name (simplified: take second field)
    parts = desc.split("|")
    species = parts[1] if len(parts) > 1 else "Unknown"

    records.append({
        "URS": urs,
        "Species": species,
        "Description": desc,
        "PDB": pdb_id
    })

# Create DataFrame
df = pd.DataFrame(records)

# Filter only entries with PDB IDs
df_pdb = df[df["PDB"].notna()].reset_index(drop=True)

print("Entries with PDB IDs:")
print(df_pdb[["URS", "PDB", "Species"]])



Entries with PDB IDs:
                                                 URS   PDB  \
0     URS00008FED48_9606|Homo_sapiens|selenocysteine  4RQF   
1                  URS000014E598_9606|Sec_from|human  7ZJW   
2         URS0000624339_10090|Mus_musculus|tRNA(Sec)  3RG5   
3  URS000080DE1A_2320|Methanopyrus_kandleri|selen...  3ADB   

                 Species  
0           Homo_sapiens  
1               Sec_from  
2           Mus_musculus  
3  Methanopyrus_kandleri  


In [5]:
# Example selection based on known URS IDs (adjust as needed)
selection = df_pdb[df_pdb["Species"].str.contains("Homo_sapiens|Mus_musculus|Methano", case=False)]
print("Selected candidates:")
print(selection)



Selected candidates:
                                                 URS                Species  \
0     URS00008FED48_9606|Homo_sapiens|selenocysteine           Homo_sapiens   
2         URS0000624339_10090|Mus_musculus|tRNA(Sec)           Mus_musculus   
3  URS000080DE1A_2320|Methanopyrus_kandleri|selen...  Methanopyrus_kandleri   

                                         Description   PDB  
0  URS00008FED48_9606|Homo_sapiens|selenocysteine...  4RQF  
2  URS0000624339_10090|Mus_musculus|tRNA(Sec) fro...  3RG5  
3  URS000080DE1A_2320|Methanopyrus_kandleri|selen...  3ADB  


Download PDB files

In [6]:
import re
from Bio.PDB import PDBList, PDBParser
import os
import shutil

pdbl = PDBList()
pdb_dir = "../data/raw/pdbs"
parser = PDBParser(QUIET=True)

structures = {}

for idx, row in selection.iterrows():
    pdb_id = row["PDB"]
    species = row["Species"]
    # Limpia species para evitar caracteres inválidos en Windows
    safe_species = re.sub(r'[^A-Za-z0-9_.-]', '_', species)
    # Nombre amigable: PDBID_Especie.pdb
    pdb_filename = f"{pdb_id}_{safe_species}.pdb"
    pdb_filepath = os.path.join(pdb_dir, pdb_filename)
    # Descarga el archivo con el nombre por defecto
    pdb_path = pdbl.retrieve_pdb_file(pdb_id, file_format="pdb", pdir=pdb_dir)
    shutil.move(pdb_path, pdb_filepath)
    # Parsear la estructura
    structure = parser.get_structure(pdb_filename, pdb_filepath)
    structures[pdb_filename] = (structure, safe_species)

# Mostrar los primeros residuos de cada estructura con nombre
for pdb_filename, (structure, species) in structures.items():
    residues = [res.resname for res in structure.get_residues()][:10]
    print(f"{pdb_filename.split('_')[0]}-{species}: {residues}")

4RQF-Homo_sapiens: ['G', 'G', 'A', 'U', 'G', 'A', 'U', 'C', 'C', 'U']
3RG5-Mus_musculus: ['G', 'C', 'C', 'C', 'G', 'G', 'A', 'U', 'G', 'A']
3ADB-Methanopyrus_kandleri: ['HIS', 'HIS', 'MSE', 'LEU', 'ILE', 'ILE', 'LEU', 'THR', 'GLY', 'LEU']


When inspecting the residues from PDB **3ADB**, an unexpected pattern appears:  
instead of only canonical nucleotides (`A, U, G, C`) as expected for a tRNA,  
the output also shows amino acids such as `HIS`, `LEU`, or `MSE`.  

It looks like the molecule has protein components, because amino acids are present.  
Let's separate the structure into **tRNA residues** and **protein residues** by chain to confirm.


In [7]:
from Bio.PDB.Polypeptide import is_aa

def classify_chain(residues):
    """Clasify a chain as protein, RNA or other."""
    if all(is_aa(r, standard=True) for r in residues if r.id[0] == " "):
        return "protein"
    elif all(r.resname.strip() in ["A", "U", "G", "C"] for r in residues if r.id[0] == " "):
        return "tRNA"
    else:
        return "other"

for pdb_filename, (structure, species) in structures.items():
    print(f"\n🔎 Analyzing {pdb_filename} ({species})")
    for chain in structure.get_chains():
        residues = list(chain.get_residues())
        chain_type = classify_chain(residues)
        if chain_type == "tRNA":
            bases = [r.resname for r in residues[:15]]
            print(f"  Chain {chain.id}: {chain_type} ({len(residues)} bases). First: {bases}")
        else:
            print(f"  Chain {chain.id}: {chain_type} ({len(residues)} residues)")



🔎 Analyzing 4RQF_Homo_sapiens.pdb (Homo_sapiens)
  Chain C: tRNA (63 bases). First: ['G', 'G', 'A', 'U', 'G', 'A', 'U', 'C', 'C', 'U', 'C', 'A', 'G', 'U', 'G']
  Chain A: protein (453 residues)
  Chain B: protein (476 residues)

🔎 Analyzing 3RG5_Mus_musculus.pdb (Mus_musculus)
  Chain A: tRNA (165 bases). First: ['G', 'C', 'C', 'C', 'G', 'G', 'A', 'U', 'G', 'A', 'U', 'C', 'C', 'U', 'C']
  Chain B: tRNA (208 bases). First: ['G', 'C', 'C', 'C', 'G', 'G', 'A', 'U', 'G', 'A', 'U', 'C', 'C', 'U', 'C']

🔎 Analyzing 3ADB_Methanopyrus_kandleri.pdb (Methanopyrus_kandleri)
  Chain A: protein (300 residues)
  Chain B: protein (280 residues)
  Chain C: tRNA (140 bases). First: ['G', 'G', 'C', 'C', 'G', 'C', 'C', 'G', 'C', 'C', 'A', 'C', 'C', 'G', 'G']
  Chain D: tRNA (123 bases). First: ['G', 'G', 'C', 'C', 'G', 'C', 'C', 'G', 'C', 'C', 'A', 'C', 'C', 'G', 'G']


#### Structural notes on tRNASec PDB entries

Selenocysteine tRNA (tRNASec) adopts a unique structure compared to canonical tRNAs, and crystallographic studies from different organisms reveal how it is recognized and modified in the cell.  

In the archaeon *Methanopyrus kandleri*, the PDB entry **3ADB** shows tRNASec in complex with **O-phosphoseryl-tRNA kinase (PSTK)**. Chains **A** and **B** represent the kinase, while chains **C** and **D** correspond to the tRNASec molecules. This structure demonstrates that archaeal tRNASec is not observed as a free RNA but rather embedded in a **protein–RNA complex**, reflecting the essential role of PSTK in converting Ser-tRNA\[Sec\] into Sec-tRNA\[Sec\]. The RNA chains are well resolved, with canonical nucleotides (`A, U, G, C`) forming the expected structural motifs.  

In humans, the PDB entry **4RQF** corresponds to the **seryl-tRNA synthetase (SerRS)** dimer bound to a single tRNASec. Chains **A** and **B** correspond to the protein dimer, and chain **C** to the tRNA. This highlights the conserved mechanism whereby SerRS specifically recognizes the unusual tRNASec scaffold in order to charge it with serine, a prerequisite for its later conversion into selenocysteine. Again, tRNASec is not free-standing but bound within a **protein–RNA complex**.  

Finally, the mouse structure **3RG5** provides additional insight. Here, only tRNASec molecules are present (chains **A** and **B**), synthesized in vitro and crystallized without their protein partners. The presence of two RNA chains does not indicate distinct functional species but rather reflects the **crystal packing**, where two copies of the same 86-nucleotide RNA occupy the asymmetric unit. This structure, solved at 2.0 Å resolution, emphasizes the **intrinsic flexibility** of the RNA, as noted in the associated publication (Ganichkin et al., 2011, PLoS ONE).  

Alternative option if you don't want to use the fasta file generated in the Jupyter notebook `alig_sequence_prediction.ipynb` you can download by the urs_id provide in RNACentral.

```python
Example human tRNA^Sec URS ID (replace for the species or value that you want)
human_urs = "URS00001DA281"
human_taxid = 9606
url = f"https://rnacentral.org/rna/{human_urs}_{human_taxid}.fasta"
human_seq = requests.get(url).text
print(human_seq)

# Save to file
with open("../data/raw/human_trnaSec.fasta", "w") as f:
    f.write(human_seq)

## Section 3: Structural analysis of tRNASec: geometry and base-pairing

Selenocysteine tRNA (tRNASec) is known to adopt a distinctive **9/4 fold**, in contrast to the canonical **7/5 fold** of standard tRNAs.  
This unusual geometry has been linked to its ability to interact with multiple protein partners (SerRS, PSTK, SelA) and to decode the UGA stop codon as selenocysteine.  

To characterize these structural features, two complementary analyses were performed:  

1. **Global geometry** (distances and angles, geometric fingerprint of the tRNA fold).  
2. **Base-pair interactions** (canonical vs non-canonical, from curated mmCIF annotations).  

The goal is to integrate these two layers —geometry and pairing— to understand whether **tRNASec maintains stability through Watson–Crick stems while introducing conserved irregularities** that could explain its flexibility and specialized function.

### 3.1. Canonical vs non-canonical pairs


In [8]:
#Download mmCIF files and rename them

pdbl = PDBList()
pdb_dir = "../data/raw/pdbs"

for idx, row in selection.iterrows():
    pdb_id = row["PDB"].lower()  # ¡ojo! mmCIF usa lowercase
    species = row["Species"]
    safe_species = re.sub(r'[^A-Za-z0-9_.-]', '_', species)
    pdb_filename = f"{pdb_id}_{safe_species}.cif"
    pdb_filepath = os.path.join(pdb_dir, pdb_filename)

    # Descargar en formato mmCIF
    pdbl.retrieve_pdb_file(pdb_id, file_format="mmCif", pdir=pdb_dir)
    # El archivo viene como "pdbXXXX.cif" → lo renombrás
    default_name = os.path.join(pdb_dir, f"{pdb_id}.cif")
    shutil.move(default_name, pdb_filepath)




In [9]:
#Analyze base pairs from mmCIF files

import pandas as pd
from Bio.PDB.MMCIF2Dict import MMCIF2Dict
import os


pdb_dir = "../data/raw/pdbs"

def extract_base_pairs(mmcif_file):
    mmcif_dict = MMCIF2Dict(mmcif_file)
    if "_struct_conn.conn_type_id" not in mmcif_dict:
        return []
    conn_types = mmcif_dict["_struct_conn.conn_type_id"]
    comp1 = mmcif_dict["_struct_conn.ptnr1_label_comp_id"]
    res1  = mmcif_dict["_struct_conn.ptnr1_label_seq_id"]
    chain1= mmcif_dict["_struct_conn.ptnr1_label_asym_id"]
    comp2 = mmcif_dict["_struct_conn.ptnr2_label_comp_id"]
    res2  = mmcif_dict["_struct_conn.ptnr2_label_seq_id"]
    chain2= mmcif_dict["_struct_conn.ptnr2_label_asym_id"]
    details = mmcif_dict["_struct_conn.details"]
    pairs = []
    for i, ctype in enumerate(conn_types):
        if "hydrog" in ctype.lower():
            pairs.append({
                "Chain1": chain1[i],
                "Residue1": f"{comp1[i]}{res1[i]}",
                "Chain2": chain2[i],
                "Residue2": f"{comp2[i]}{res2[i]}",
                "Type": details[i]
            })
    return pairs

# Loop over all mmCIF in your folder and show as table
for fname in os.listdir(pdb_dir):
    if fname.endswith(".cif") or fname.endswith(".mmcif"):
        cif_path = os.path.join(pdb_dir, fname)
        pairs = extract_base_pairs(cif_path)
        print(f"\n📄 Base pairs for {fname}:")
        if pairs:
            df = pd.DataFrame(pairs)
            display(df)  # Jupyter: muestra como tabla
        else:
            print("No base-pair annotations found.")


📄 Base pairs for 3adb_Methanopyrus_kandleri.cif:


,Chain1,Residue1,Chain2,Residue2,Type
0,C,G1,C,C88,WATSON-CRICK
1,C,G1,C,C88,WATSON-CRICK
2,C,G1,C,C88,WATSON-CRICK
3,C,G2,C,C87,WATSON-CRICK
4,C,G2,C,C87,WATSON-CRICK
...,...,...,...,...,...
207,D,G68,D,C76,WATSON-CRICK
208,D,G68,D,C76,WATSON-CRICK
209,D,G68,D,C76,WATSON-CRICK
210,D,U69,D,A73,REVERSED HOOGSTEEN



📄 Base pairs for 3rg5_Mus_musculus.cif:


,Chain1,Residue1,Chain2,Residue2,Type
0,A,G1,A,C86,WATSON-CRICK
1,A,G1,A,C86,WATSON-CRICK
2,A,G1,A,C86,WATSON-CRICK
3,A,C2,A,G85,WATSON-CRICK
4,A,C2,A,G85,WATSON-CRICK
...,...,...,...,...,...
176,B,G66,B,C74,WATSON-CRICK
177,B,G66,B,C74,WATSON-CRICK
178,B,G66,B,C74,WATSON-CRICK
179,B,U67,B,A71,REVERSED HOOGSTEEN



📄 Base pairs for 4rqf_Homo_sapiens.cif:


,Chain1,Residue1,Chain2,Residue2,Type
0,A,G6,A,U81,G-U MISPAIR
1,A,A7,A,U80,WATSON-CRICK
2,A,A7,A,U80,WATSON-CRICK
3,A,U8,A,U79,TYPE_16_PAIR
4,A,U8,A,U79,TYPE_16_PAIR
5,A,G9,A,C78,WATSON-CRICK
6,A,G9,A,C78,WATSON-CRICK
7,A,G9,A,C78,WATSON-CRICK
8,A,C12,A,G27,WATSON-CRICK
9,A,C12,A,G27,WATSON-CRICK


In [10]:
# Automated Base Pair Report Generator for Structural Biology Files

# Crear carpeta de resultados si no existe
out_dir = "../results/tables"
os.makedirs(out_dir, exist_ok=True)

all_pairs = []

for fname in os.listdir(pdb_dir):
    if fname.endswith(".cif") or fname.endswith(".mmcif"):
        cif_path = os.path.join(pdb_dir, fname)
        pairs = extract_base_pairs(cif_path)  # usa tu función previa
        for p in pairs:
            p["PDB"] = fname.split("_")[0].upper()
            all_pairs.append(p)

df_all = pd.DataFrame(all_pairs)

# Resumen por PDB y tipo de interacción
summary = df_all.groupby(["PDB","Type"]).size().reset_index(name="Count")

# Totales por PDB
total = df_all.groupby("PDB").size().reset_index(name="TotalPairs")

# Mostrar en la notebook
for pdb in summary["PDB"].unique():
    print(f"\n🔎 Base-pair types for {pdb}:")
    display(summary[summary["PDB"] == pdb])

print("\n📊 Total number of annotated base pairs:")
display(total)

# Guardar en un único CSV concatenado
report_path = os.path.join(out_dir, "basepair_report.csv")

with open(report_path, "w") as f:
    f.write("### Base-pair types by PDB\n")
    summary.to_csv(f, index=False)
    f.write("\n### Total number of annotated base pairs\n")
    total.to_csv(f, index=False)

print(f"\n✅ Report saved as {report_path}")



🔎 Base-pair types for 3ADB:


,PDB,Type,Count
0,3ADB,G-U MISPAIR,4
1,3ADB,REVERSED HOOGSTEEN,4
2,3ADB,TYPE_13_PAIR,4
3,3ADB,TYPE_25_PAIR,2
4,3ADB,TYPE_26_PAIR,2
5,3ADB,WATSON-CRICK,196



🔎 Base-pair types for 3RG5:


,PDB,Type,Count
6,3RG5,C-A MISPAIR,2
7,3RG5,C-G PAIR,1
8,3RG5,G-U MISPAIR,4
9,3RG5,REVERSED HOOGSTEEN,4
10,3RG5,REVERSED WATSON-CRICK,2
11,3RG5,TYPE_13_PAIR,4
12,3RG5,TYPE_16_PAIR,4
13,3RG5,TYPE_28_PAIR,16
14,3RG5,TYPE_8_PAIR,2
15,3RG5,U-A PAIR,1



🔎 Base-pair types for 4RQF:


,PDB,Type,Count
18,4RQF,G-A MISPAIR,1
19,4RQF,G-C PAIR,1
20,4RQF,G-U MISPAIR,3
21,4RQF,REVERSED HOOGSTEEN,2
22,4RQF,TYPE_13_PAIR,2
23,4RQF,TYPE_16_PAIR,2
24,4RQF,TYPE_28_PAIR,2
25,4RQF,TYPE_8_PAIR,2
26,4RQF,U-A PAIR,1
27,4RQF,U-C MISPAIR,1



📊 Total number of annotated base pairs:


,PDB,TotalPairs
0,3ADB,212
1,3RG5,181
2,4RQF,60



✅ Report saved as ../results/tables\basepair_report.csv


The mmCIF base-pair annotations reveal that tRNASec is stabilized by a Watson–Crick backbone yet incorporates a substantial fraction of non-canonical interactions. In the archaeal structure (3ADB), 212 total pairs were annotated, of which 196 (92.5%) are Watson–Crick and 16 are non-canonical, including several G–U wobbles. The murine structure (3RG5) shows 181 pairs, with 139 Watson–Crick and a notably higher 42 non-canonical contacts (23.2%), spanning G–U, C–A, and U–C mismatches. The human structure (4RQF) contains only 60 annotated pairs but exhibits the highest proportion of non-canonical interactions: 17 (28.3%) involve G–A, G–U, or U–C mispairs. Compared to canonical tRNAs, where mismatches are rare, this elevated and diverse set of non-canonical interactions appears as a conserved hallmark of tRNASec. Taken together, these findings support the idea that tRNASec has evolved a **dual structural strategy**: canonical Watson–Crick pairs provide overall stability, while a high proportion of non-canonical contacts strategically introduce conformational versatility, enabling recognition by multiple protein partners and underpinning its unique role in selenocysteine incorporation.



###  3.2. Stem-specific base-pair counts


Beyond listing canonical vs. non-canonical pairs, it is informative to ask **how many pairs belong to each stem of the cloverleaf structure** (acceptor, D, anticodon, T, variable).  

This provides a direct way to confirm the **elongation of specific stems** that distinguishes tRNASec from canonical tRNAs:  
- The **acceptor + T domain** is extended (9/4 or 8/5 instead of the canonical 7/5).  
- The **D-stem** is unusually long (6–7 bp).  
- The **anticodon stem** remains canonical (5 bp).  
- The **variable arm** forms an extra mini-helix (4–6 bp).  

By mapping base pairs to residue ranges defined for each stem, we can automatically classify and count them from the mmCIF annotations.  
This yields a reproducible description of the “7/5 vs 9/4” distinction that is otherwise qualitative in the literature.

In [ ]:
def count_stem_pairs(cif_path):
    """
    Parse mmCIF (_struct_conn records) y devuelve DataFrame con:
    Stem, Type, Residues.
    """
    import gemmi
    import pandas as pd

    doc = gemmi.cif.read_file(cif_path)
    block = doc.sole_block()
    conn = block.find_mmcif_category('_struct_conn')

    basepairs = []
    for row in conn:
        nt1 = f"{row['_struct_conn.ptnr1_auth_comp_id']}{row['_struct_conn.ptnr1_auth_seq_id']}"
        nt2 = f"{row['_struct_conn.ptnr2_auth_comp_id']}{row['_struct_conn.ptnr2_auth_seq_id']}"
        pair_type = row.get('_struct_conn.conn_type_id', 'Unknown')

        # asignar regiones según número de nt
        def region(res):
            try:
                num = int(''.join([c for c in res if c.isdigit()]))
            except ValueError:
                return "Other"
            if num <= 7 or num >= 66:
                return "Acceptor"
            elif 8 <= num <= 19:
                return "D"
            elif 27 <= num <= 43:
                return "Anticodon"
            elif 44 <= num <= 65:
                return "T"
            elif 20 <= num <= 26:
                return "Variable"
            else:
                return "Other"

        r1 = region(nt1)
        r2 = region(nt2)
        stem = r1 if r1 == r2 else f"{r1}-{r2}"

        basepairs.append({
            "Stem": stem,
            "Type": pair_type.upper(),
            "Residues": f"{nt1}-{nt2}"
        })

    return pd.DataFrame(basepairs)




In [33]:
import pandas as pd
from Bio.PDB.MMCIF2Dict import MMCIF2Dict

def count_stem_pairs(mmcif_file):
    """
    Extrae pares de bases (hydrogen-bonded) de un archivo mmCIF y
    devuelve un DataFrame con columnas: Stem, Residue1, Residue2, Type.
    """
    mmcif_dict = MMCIF2Dict(mmcif_file)
    if "_struct_conn.conn_type_id" not in mmcif_dict:
        return pd.DataFrame()

    conn_types = mmcif_dict["_struct_conn.conn_type_id"]
    comp1  = mmcif_dict["_struct_conn.ptnr1_label_comp_id"]
    res1   = mmcif_dict["_struct_conn.ptnr1_label_seq_id"]
    chain1 = mmcif_dict["_struct_conn.ptnr1_label_asym_id"]
    comp2  = mmcif_dict["_struct_conn.ptnr2_label_comp_id"]
    res2   = mmcif_dict["_struct_conn.ptnr2_label_seq_id"]
    chain2 = mmcif_dict["_struct_conn.ptnr2_label_asym_id"]
    details= mmcif_dict.get("_struct_conn.details", ["NA"]*len(conn_types))

    pairs = []
    for i, ctype in enumerate(conn_types):
        if "hydrog" in ctype.lower():   # ⚡ filtra solo interacciones por H-bond
            res1_str = f"{chain1[i]}:{comp1[i]}{res1[i]}"
            res2_str = f"{chain2[i]}:{comp2[i]}{res2[i]}"
            stem_id = f"{chain1[i]}-{chain2[i]}"
            pairs.append({
                "Stem": stem_id,
                "Residue1": res1_str,
                "Residue2": res2_str,
                "Type": details[i]
            })

    return pd.DataFrame(pairs)




In [37]:
# Crear carpeta de resultados si no existe
out_dir = "../results/tables"
os.makedirs(out_dir, exist_ok=True)

all_stem_data = []

for fname in os.listdir(pdb_dir):
    if fname.endswith(".cif") or fname.endswith(".mmcif"):
        cif_path = os.path.join(pdb_dir, fname)
        df_stems = count_stem_pairs(cif_path)
        if not df_stems.empty:
            df_stems["PDB"] = fname.split("_")[0].upper()
            all_stem_data.append(df_stems)

# Concatenar todos los resultados
if all_stem_data:
    df_all_stems = pd.concat(all_stem_data, ignore_index=True)

    # Resumen por PDB, tallo y tipo de interacción
    summary_stems = (
        df_all_stems.groupby(["PDB", "Stem", "Type"])
        .size()
        .reset_index(name="Count")
    )

    # Guardar como CSV
    stem_report_path = os.path.join(out_dir, "basepair_by_stem.csv")
    summary_stems.to_csv(stem_report_path, index=False, mode="w")


    print(f"\n✅ Reports saved:\n- {stem_report_path}")
else:
    print("⚠️ No stem-specific base pairs found in the analyzed files.")



✅ Reports saved:
- ../results/tables\basepair_by_stem.csv


In [26]:
df = pd.read_csv(cif_path, sep="\t", comment="#")
print(df.columns.tolist())
print(df.head())

['data_3ADB']
                                           data_3ADB
0                                  _entry.id   3ADB 
1     _audit_conform.dict_name       mmcif_pdbx.dic 
2              _audit_conform.dict_version    5.397 
3  _audit_conform.dict_location   http://mmcif.pd...
4                                              loop_


Across archaeal (3ADB), murine (3RG5), and human (4RQF) tRNASec, the stem-specific base-pair analysis reveals a clear pattern: while the D- and anticodon-stems are largely stabilized by canonical Watson–Crick pairs, the T-stem consistently incorporates **non-canonical `REVERSED HOOGSTEEN` interactions**. This recurrent feature across species highlights that the structural versatility of tRNASec is not random but rather an **evolutionarily conserved trait**, in which canonical stems provide backbone stability and selected non-canonical pairs introduce localized flexibility essential for its unique biological function.


## Section 4: 3D visualization of tRNASec structures

To complement the quantitative analysis, the three representative tRNASec structures were explored in **3D**.  
This serves two purposes:  

1. **Visual inspection of global geometry**  
   - Confirm the characteristic **L-shape**.  
   - Highlight the elongated **acceptor–T domain** (9/4 fold).  
   - Compare compactness vs. openness of the elbow region across species.  

2. **Base-pair mapping in 3D**  
   - Use curated annotations from mmCIF (`_struct_conn`) to highlight **Watson–Crick pairs** in the 3D viewer.  
   - Visually differentiate canonical stems (colored in one scheme) vs. **non-canonical pairs** (highlighted separately).  
   - This allows intuitive recognition of where flexibility is introduced.

For interactive visualization, the library **py3Dmol** was used directly in the notebook.  


In [12]:
import os
import py3Dmol
import ipywidgets as widgets
from IPython.display import display
from Bio.PDB import MMCIFParser

# ---- 1. Show only tRNASec  ----

def detect_tRNA_chains(cif_file):
    """
    Return a list of chain IDs classified as tRNA.
    """
    parser = MMCIFParser(QUIET=True)
    structure = parser.get_structure("model", cif_file)
    tRNA_chains = []
    for chain in structure.get_chains():
        residues = list(chain.get_residues())
        if len(residues) == 0:
            continue
        if classify_chain(residues) == "tRNA":
            tRNA_chains.append(chain.id)
    return tRNA_chains


def show_tRNA_dropdown(folder_path="../data/raw/pdbs/"):
    """
    Dropdown menu to visualize ONLY one tRNA chain per structure,
    using the existing classify_chain function.
    If multiple tRNAs exist, shows the first one.
    """
    files = [f for f in os.listdir(folder_path) if f.endswith(".cif")]
    dropdown = widgets.Dropdown(options=files, description="Select file:")
    output = widgets.Output()

    def visualize(change):
        output.clear_output()
        fname = change["new"]
        fpath = os.path.join(folder_path, fname)

        # use your existing logic to detect tRNA chains
        tRNA_chains = detect_tRNA_chains(fpath)
        if not tRNA_chains:
            with output:
                print(f"No tRNA chain detected in {fname}")
            return

        # take the first tRNA chain
        chain_id = tRNA_chains[0]

        with open(fpath, "r") as f:
            data = f.read()

        view = py3Dmol.view(width=500, height=500)
        view.addModel(data, "CIF")
        view.setStyle({}, {})  # reset
        view.setStyle({'chain': chain_id}, {'cartoon': {'color': 'spectrum'}})
        view.zoomTo({'chain': chain_id})

        with output:
            print(f"Showing tRNA chain {chain_id} from {fname}")
            display(view.show())

    dropdown.observe(visualize, names="value")
    display(dropdown, output)
    visualize({"new": files[0]})  # load first file initially


# Example usage
show_tRNA_dropdown("../data/raw/pdbs/")


Dropdown(description='Select file:', options=('3adb_Methanopyrus_kandleri.cif', '3rg5_Mus_musculus.cif', '4rqf…

Output()

In [ ]:
import os
import py3Dmol
import ipywidgets as widgets
from IPython.display import display
from Bio.PDB import MMCIFParser

# ---- 2. Show tRNASec + protein  ----


def detect_chain_types(cif_file):
    """Return a dict {chain_id: 'protein' | 'tRNA' | 'other'}"""
    parser = MMCIFParser(QUIET=True)
    structure = parser.get_structure("model", cif_file)
    chain_types = {}
    for chain in structure.get_chains():
        residues = list(chain.get_residues())
        if len(residues) == 0:
            continue
        chain_types[chain.id] = classify_chain(residues)  # your existing classify_chain
    return chain_types


def show_one_protein_one_tRNA(folder_path="../data/raw/pdbs/"):
    """
    Dropdown menu to visualize exactly ONE protein + ONE tRNA from each structure.
    Cleans styles before drawing to avoid ghost chains.
    """
    files = [f for f in os.listdir(folder_path) if f.endswith(".cif")]
    dropdown = widgets.Dropdown(options=files, description="Select file:")
    output = widgets.Output()

    def visualize(change):
        output.clear_output()
        fname = change["new"]
        fpath = os.path.join(folder_path, fname)

        chain_types = detect_chain_types(fpath)
        if not chain_types:
            with output:
                print(f"No valid chains in {fname}")
            return

        with open(fpath, "r") as f:
            data = f.read()

        view = py3Dmol.view(width=600, height=500)
        view.addModel(data, "CIF")

        # 🔹 Reset styles
        view.setStyle({}, {})

        protein_chain = None
        tRNA_chain = None

        # Pick the FIRST protein and FIRST tRNA only
        for chain_id, ctype in chain_types.items():
            if ctype == "protein" and protein_chain is None:
                protein_chain = chain_id
                # Use a stronger color so it’s clearly visible
                view.setStyle({'chain': protein_chain}, {'cartoon': {'color': 'lightblue'}})
            elif ctype == "tRNA" and tRNA_chain is None:
                tRNA_chain = chain_id
                view.setStyle({'chain': tRNA_chain}, {'cartoon': {'color': 'spectrum'}})

        view.zoomTo()

        with output:
            if protein_chain and tRNA_chain:
                print(f"Showing protein chain {protein_chain} + tRNA chain {tRNA_chain} from {fname}")
            elif protein_chain:
                print(f"Showing only protein chain {protein_chain} from {fname}")
            elif tRNA_chain:
                print(f"Showing only tRNA chain {tRNA_chain} from {fname}")
            else:
                print(f"No protein or tRNA found in {fname}")
            display(view.show())

    dropdown.observe(visualize, names="value")
    display(dropdown, output)
    visualize({"new": files[0]})


# Example usage
show_one_protein_one_tRNA("../data/raw/pdbs/")



Dropdown(description='Select file:', options=('3adb_Methanopyrus_kandleri.cif', '3rg5_Mus_musculus.cif', '4rqf…

Output()

In [ ]:
import os
import py3Dmol
import ipywidgets as widgets
from IPython.display import display
from Bio.PDB.MMCIF2Dict import MMCIF2Dict

# --- Detectar residuos no canónicos desde _struct_conn ---
def detect_key_residues_from_cif(pdb_id, cif_file, summary_stems):
    mmcif_dict = MMCIF2Dict(cif_file)

    if "_struct_conn.conn_type_id" not in mmcif_dict:
        return {}

    conn_types = mmcif_dict["_struct_conn.conn_type_id"]
    comp1  = mmcif_dict["_struct_conn.ptnr1_label_comp_id"]
    res1   = mmcif_dict["_struct_conn.ptnr1_label_seq_id"]
    chain1 = mmcif_dict["_struct_conn.ptnr1_label_asym_id"]
    comp2  = mmcif_dict["_struct_conn.ptnr2_label_comp_id"]
    res2   = mmcif_dict["_struct_conn.ptnr2_label_seq_id"]
    chain2 = mmcif_dict["_struct_conn.ptnr2_label_asym_id"]
    details= mmcif_dict["_struct_conn.details"]

    # normalizar a listas
    if isinstance(conn_types, str):
        conn_types = [conn_types]
        comp1, res1, chain1 = [comp1],[res1],[chain1]
        comp2, res2, chain2 = [comp2],[res2],[chain2]
        details = [details]

    residues_to_highlight = {}
    for c1, r1, c2, r2, t, d in zip(chain1, res1, chain2, res2, conn_types, details):
        if "hydrog" in t.lower():  # pares por puente de H
            if any(key in d.upper() for key in ["MISPAIR","HOOGSTEEN","TYPE"]):
                residues_to_highlight.setdefault(c1, []).append(int(r1))
                residues_to_highlight.setdefault(c2, []).append(int(r2))

    return {ch: sorted(set(res)) for ch,res in residues_to_highlight.items()}


# --- Visualizador con dropdown ---
def show_protein_tRNA_dropdown(data_dir, summary_stems):
    files = [f for f in os.listdir(data_dir) if f.endswith(".cif")]
    dropdown = widgets.Dropdown(options=files, description="Select file:")
    output = widgets.Output()

    def visualize(change):
        output.clear_output()
        fname = change["new"]
        filepath = os.path.join(data_dir, fname)

        pdb_id = fname.split("_")[0].upper()
        residues_interest = detect_key_residues_from_cif(pdb_id, filepath, summary_stems)

        view = py3Dmol.view(width=600, height=500)
        view.addModel(open(filepath).read(), "mmcif")

        # reset estilos
        view.setStyle({}, {})

        # --- elegir UNA proteína (la más larga por nº de residuos, o A si no hay info) ---
        protein_chain = None
        rna_chain = None

        # buscar cadenas que tienen anomalías (residues_interest → tRNA)
        if residues_interest:
            rna_chain = list(residues_interest.keys())[0]  # solo la primera cadena de RNA detectada

        # heurística: usar cadena A como proteína (si existe)
        # podés cambiar esta regla si querés otra lógica
        protein_chain = "A"

        # --- dibujar proteína en gris ---
        if protein_chain:
            view.setStyle({'chain': protein_chain}, {'cartoon': {'color': 'lightgrey'}})

        # --- dibujar tRNA en gris + sticks de colores ---
        if rna_chain:
            view.setStyle({'chain': rna_chain}, {'cartoon': {'color': 'lightgrey'}})

            colors = ['lightgreen','darkmagenta','orange','cyan']
            for i, resi in enumerate(residues_interest.get(rna_chain, [])):
                view.setStyle({'chain': rna_chain, 'resi': str(resi)},
                              {'stick': {'color': colors[i % len(colors)], 'radius': 0.35}})

        view.setBackgroundColor('white')
        view.zoomTo()

        with output:
            print(f"Showing {fname}")
            if protein_chain:
                print(f"Protein chain: {protein_chain}")
            if rna_chain:
                print(f"tRNA chain: {rna_chain} | residues: {residues_interest.get(rna_chain, [])}")
            display(view.show())

    dropdown.observe(visualize, names="value")
    display(dropdown, output)

    if files:
        visualize({"new": files[0]})

data_dir = "../data/raw/pdbs/"
show_protein_tRNA_dropdown(data_dir, summary_stems)


Dropdown(description='Select file:', options=('3adb_Methanopyrus_kandleri.cif', '3rg5_Mus_musculus.cif', '4rqf…

Output()

In [39]:
import os
import py3Dmol
import ipywidgets as widgets
from IPython.display import display
from Bio.PDB.MMCIF2Dict import MMCIF2Dict

# --- Detect non-canonical residues from _struct_conn ---
def detect_key_residues_from_cif(pdb_id, cif_file, summary_stems):
    """
    Parse an mmCIF file to detect key RNA residues involved in non-canonical interactions 
    (e.g., mismatches, Hoogsteen, alternative pair types).
    
    Parameters
    ----------
    pdb_id : str
        Identifier of the PDB entry.
    cif_file : str
        Path to the mmCIF file.
    summary_stems : DataFrame
        Summary of base-pair interactions (not directly used here, but can be extended).

    Returns
    -------
    dict
        Dictionary with chain IDs as keys and lists of residue indices to highlight.
    """
    mmcif_dict = MMCIF2Dict(cif_file)

    if "_struct_conn.conn_type_id" not in mmcif_dict:
        return {}

    # Extract mmCIF fields
    conn_types = mmcif_dict["_struct_conn.conn_type_id"]
    comp1  = mmcif_dict["_struct_conn.ptnr1_label_comp_id"]
    res1   = mmcif_dict["_struct_conn.ptnr1_label_seq_id"]
    chain1 = mmcif_dict["_struct_conn.ptnr1_label_asym_id"]
    comp2  = mmcif_dict["_struct_conn.ptnr2_label_comp_id"]
    res2   = mmcif_dict["_struct_conn.ptnr2_label_seq_id"]
    chain2 = mmcif_dict["_struct_conn.ptnr2_label_asym_id"]
    details= mmcif_dict["_struct_conn.details"]

    # Normalize to lists (Bio.PDB sometimes returns strings if only one entry exists)
    if isinstance(conn_types, str):
        conn_types = [conn_types]
        comp1, res1, chain1 = [comp1], [res1], [chain1]
        comp2, res2, chain2 = [comp2], [res2], [chain2]
        details = [details]

    residues_to_highlight = {}
    for c1, r1, c2, r2, t, d in zip(chain1, res1, chain2, res2, conn_types, details):
        # Filter hydrogen-bonded interactions
        if "hydrog" in t.lower():
            # Detect annotations of interest (mispair, Hoogsteen, or type variations)
            if any(key in d.upper() for key in ["MISPAIR", "HOOGSTEEN", "TYPE"]):
                residues_to_highlight.setdefault(c1, []).append(int(r1))
                residues_to_highlight.setdefault(c2, []).append(int(r2))

    # Remove duplicates and sort residues within each chain
    return {ch: sorted(set(res)) for ch, res in residues_to_highlight.items()}


# --- Interactive visualization with dropdown ---
def show_protein_tRNA_dropdown(data_dir, summary_stems):
    """
    Build an interactive widget to explore protein–tRNA structures.
    For each mmCIF file in the given directory, highlight key residues 
    detected from _struct_conn annotations.

    Parameters
    ----------
    data_dir : str
        Path to the directory containing mmCIF files.
    summary_stems : DataFrame
        DataFrame summarizing base-pair interactions.
    """
    files = [f for f in os.listdir(data_dir) if f.endswith(".cif")]
    dropdown = widgets.Dropdown(options=files, description="Select file:")
    output = widgets.Output()

    def visualize(change):
        output.clear_output()
        fname = change["new"]
        filepath = os.path.join(data_dir, fname)

        pdb_id = fname.split("_")[0].upper()
        residues_interest = detect_key_residues_from_cif(pdb_id, filepath, summary_stems)

        # Initialize 3D viewer
        view = py3Dmol.view(width=600, height=500)
        view.addModel(open(filepath).read(), "mmcif")

        # Reset styles
        view.setStyle({}, {})

        # Heuristic: choose protein chain "A" by default
        protein_chain = "A"
        rna_chain = None

        # If anomalous residues detected, take the first RNA chain as representative
        if residues_interest:
            rna_chain = list(residues_interest.keys())[0]

        # --- Draw protein in light grey cartoon ---
        if protein_chain:
            view.setStyle({'chain': protein_chain}, {'cartoon': {'color': 'lightgrey'}})

        # --- Draw RNA in light grey cartoon + colored sticks for key residues ---
        if rna_chain:
            view.setStyle({'chain': rna_chain}, {'cartoon': {'color': 'lightgrey'}})

            colors = ['lightgreen', 'darkmagenta', 'orange', 'cyan']
            for i, resi in enumerate(residues_interest.get(rna_chain, [])):
                view.setStyle(
                    {'chain': rna_chain, 'resi': str(resi)},
                    {'stick': {'color': colors[i % len(colors)], 'radius': 0.35}}
                )

        # Set white background and zoom to fit
        view.setBackgroundColor('white')
        view.zoomTo()

        # Print context info and display viewer
        with output:
            print(f"Showing {fname}")
            if protein_chain:
                print(f"Protein chain: {protein_chain}")
            if rna_chain:
                print(f"tRNA chain: {rna_chain} | residues: {residues_interest.get(rna_chain, [])}")
            display(view.show())

    # Link dropdown selection to visualization
    dropdown.observe(visualize, names="value")
    display(dropdown, output)

    # Auto-load first file if available
    if files:
        visualize({"new": files[0]})


# Example usage
data_dir = "../data/raw/pdbs/"
show_protein_tRNA_dropdown(data_dir, summary_stems)


Dropdown(description='Select file:', options=('3adb_Methanopyrus_kandleri.cif', '3rg5_Mus_musculus.cif', '4rqf…

Output()

## Section 5: 🧬 Molecular Dynamics of tRNA–Protein Complex with OpenMM

_soon will be ready_ 


import openmm as mm
from openmm import unit, app
from openmm.app import PDBFile

```python 
# 1. Leer el PDB ya corregido
pdb = PDBFile("3RG5_fixed.pdb")

# 2. Cargar forcefield para corregir formato pdb
# Usa amber14 (o amber19) + TIP3P para RNA
forcefield = app.ForceField("amber14-all.xml", "tip3p.xml")

# 3. Crear sistema
system = forcefield.createSystem(
    pdb.topology,
    nonbondedMethod=app.PME,
    nonbondedCutoff=1*unit.nanometer,
    constraints=app.HBonds
)

# 4. Definir integrador
integrator = mm.LangevinIntegrator(
    300*unit.kelvin,   # temperatura
    1/unit.picoseconds, 
    0.002*unit.picoseconds   # timestep = 2 fs
)

# 5. Crear simulación
simulation = app.Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)

# 6. Minimización de energía
print("🔹 Minimizing energy...")
simulation.minimizeEnergy()

# 7. Reporters (para guardar datos de la simulación)
simulation.reporters.append(
    app.StateDataReporter("output.log", 100, step=True, potentialEnergy=True, temperature=True)
)
simulation.reporters.append(app.DCDReporter("trajectory.dcd", 1000))

# 8. Simulación (ej. 50 ps)
print("🔹 Running short simulation...")
simulation.step(25000)  # 25,000 pasos * 2 fs = 50 ps
